In [ ]:
import pandas as pd
from sklearn.metrics.cluster import silhouette_score, silhouette_samples
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from master_thesis.hsbmpy import colors_cycle

In [ ]:
#GTEx
df = pd.read_csv('https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz', skiprows=2, compression='gzip', sep='\t')
df['ensg'] = [x[:15] for x in df['Name']]
df.set_index('Name', inplace=True)
df.set_index(['ensg'],inplace=True)
df=df.drop(['Description'],1)
genelist=pd.read_csv("https://stephenslab.github.io/count-clustering/project/utilities/gene_names_all_gtex.txt", header=None).values.ravel()
df = df[df.index.isin(genelist)]
df_file = pd.read_csv("https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt", sep='\t').loc[:,['SAMPID','SMTS', 'SMTSD']]
df_file.set_index('SAMPID', inplace=True)

In [ ]:
#TCGA
df=pd.read_csv("datasets/tcga/oversampling_10tissue/mainTable_all.csv", index_col=0)
df_file=pd.read_csv("datasets/tcga/oversampling_10tissue/files.dat", index_col=0)

In [ ]:
import os
os.chdir("datasets/tcga/oversampling_10tissue/")

In [ ]:
files = (file for file in df_file.index if file in df.columns)

In [ ]:
df = df[list(files)]
df.head()

In [ ]:
df = df.sample(n=3000, axis=1, random_state=42).sample(5000, axis=0, random_state=42)

In [ ]:
data = df.transpose().values
data[np.isnan(data)]=0
files=df.columns
genes=df.index
del df

In [ ]:
df_file.columns

In [ ]:
classes, cluster_labels = np.unique([df_file.at[sample, 'SMTS'] for sample in files], return_inverse=True)
n_clusters = 1+cluster_labels.max()

In [ ]:
data = np.log2(data+1)
data[np.isinf(data)]=np.log2(1e6+1)
#np.random.shuffle(cluster_labels)

In [ ]:
data.shape

In [ ]:
from telepyth import TelepythClient
tc=TelepythClient('8884910787382816523')

In [ ]:
metrics = ['euclidean', 'cosine']
label = 'pca'
#k='euclidean'
def make_silhouette(datas, labels):
    for data,label in zip(datas, labels):
        for k in metrics:
            y_lower = 10
            sample_silhouette_values = silhouette_samples(data, cluster_labels, metric=k)
            silhouette_avg=silhouette_score(data, cluster_labels, metric=k)
            fig, ax = plt.subplots(1, 1)
            fig.set_size_inches(15, 35)

            # The 1st subplot is the silhouette plot
            # The silhouette coefficient can range from -1, 1 but in this example all
            # lie within [-0.1, 1]
            ax.set_xlim([-1, 1])
            # The (n_clusters+1)*10 is for inserting blank space between silhouette
            # plots of individual clusters, to demarcate them clearly.
            ax.set_ylim([0, data.shape[0] + (n_clusters + 1) * 10])

            for i in range(n_clusters):
                # cluster i, and sort them
                ith_cluster_silhouette_values = \
                    sample_silhouette_values[cluster_labels == i]

                ith_cluster_silhouette_values.sort()

                size_cluster_i = ith_cluster_silhouette_values.shape[0]
                y_upper = y_lower + size_cluster_i

                color = cm.gnuplot(float(i) / n_clusters)
                ax.fill_betweenx(np.arange(y_lower, y_upper),
                                  0, ith_cluster_silhouette_values,
                                  facecolor=color, edgecolor=color, alpha=0.7)

                ax.text(-0.8, (y_lower+y_upper)/2, "%s"%classes[i], fontsize=18)

                # Compute the new y_lower for next plot
                y_lower = y_upper + 10  # 10 for the 0 samples


            ax.set_title("gtex %s\n%s metric\n%d clusters\n%s space"%(label,k,n_clusters, label), fontsize=20)
            ax.set_xlabel("score", fontsize=20)
            # Label the silhouette plots with their cluster numbers at the middle
            ax.axvline(x=silhouette_avg, color="red", linestyle="--", lw=2)
            plt.tick_params(labelsize=20)
            plt.show()
            fig.savefig("silhouette_gtex_%s_%s.pdf"%(label,k))
            del sample_silhouette_values
            del silhouette_avg
            tc.send_figure(fig)

In [ ]:
print(red_data.shape)
print(lda_data.shape)
print(emb_data.shape)
print(hsbm_data.shape)
print(tm_data.shape)
make_silhouette([data, red_data, lda_data, emb_data, hsbm_data, tm_data],
               ['data', 'pca', 'lda', 'tsne', 'hsbm', 'tm'])

## PCA

In [ ]:
from sklearn.manifold import MDS
from sklearn.decomposition import PCA

In [ ]:
model=PCA(n_components=2)

In [ ]:
red_data=model.fit_transform(data)

In [ ]:
#%matplotlib notebook
fig=plt.figure(figsize=(15,15))
for i in range(cluster_labels.max()+1):
    plt.scatter(red_data[cluster_labels==i].T[0],red_data[cluster_labels==i].T[1], label=classes[i], c=colors_cycle[i])
plt.legend(ncol=4, fontsize=10)
#plt.xlim(red_data.T[0].min(),red_data.T[0].max())
plt.tick_params(labelsize=20)
plt.show()
fig.savefig("mds_02.pdf")

In [ ]:
import pickle

In [ ]:
emb_datawith open("pca_model.pkl",'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open("pca_model.pkl",'rb') as f:
    model = pickle.load(f)

In [ ]:
red_data = model.transform(data)

## kmeans

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
model = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)

In [ ]:
cluster_labels = model.fit_predict(data)
classes = np.unique(cluster_labels)

## hierarchical

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc

In [ ]:
fig=plt.figure()
Z = shc.linkage(data, method='average')
dend = shc.dendrogram(Z, leaf_rotation=90., leaf_font_size=8.,)
plt.xlabel("samples", fontsize=16)
plt.show()
fig.savefig("dendogram.pdf")

In [ ]:
cut = shc.cut_tree(Z, height=20000).ravel()
np.unique(cut, return_counts=True)

In [ ]:
model = AgglomerativeClustering(n_clusters=n_clusters, affinity='cosine', linkage='complete')

In [ ]:
cluster_labels = model.fit_predict(data[cut==0])
classes = np.unique(cluster_labels)

In [ ]:
out_to_file(cluster_labels, files[cut==0], name='hier')

## LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
lda = LatentDirichletAllocation(n_components=n_clusters)

In [ ]:
lda_data = lda.fit_transform(data)
cluster_labels = np.argmax(lda_data, axis=1)
classes = np.arange(0, np.max(cluster_labels)+1)

In [ ]:
fig=plt.figure(figsize=(15,15))
for i in range(cluster_labels.max()+1):
    plt.scatter(lda_data[cluster_labels==i].T[3],lda_data[cluster_labels==i].T[4], label=classes[i], c=colors_cycle[i])
plt.legend(ncol=4, fontsize=10)
#plt.xlim(red_data.T[0].min(),red_data.T[0].max())
plt.tick_params(labelsize=20)
plt.show()
fig.savefig("lda_02.pdf")

## hSBM

In [ ]:
from master_thesis.hsbm.sbmtm import sbmtm

In [ ]:
hsbm = sbmtm()

In [ ]:
hsbm.make_graph_from_BoW_df(pd.DataFrame(data=data))

In [ ]:
hsbm.save_graph("graph.xml.gz")

In [ ]:
hsbm.fit()

In [ ]:
hsbm_data = pd.read_csv("topsbm/topsbm_level_3_topic-dist.csv").set_index('doc').reindex(index=[s[:24] for s in files]).drop('i_doc',1).fillna(0).values

In [ ]:
fig=plt.figure(figsize=(15,15))
for i in range(cluster_labels.max()+1):
    plt.scatter(hsbm_data[cluster_labels==i].T[0],hsbm_data[cluster_labels==i].T[1], label=classes[i], c=colors_cycle[i])
plt.legend(ncol=4, fontsize=10)
#plt.xlim(red_data.T[0].min(),red_data.T[0].max())
plt.tick_params(labelsize=20)
plt.show()
fig.savefig("hsbm_02.pdf")

## t-SNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2)

In [ ]:
emb_data = tsne.fit_transform(data)

In [ ]:
fig=plt.figure(figsize=(15,15))
for i in range(cluster_labels.max()+1):
    plt.scatter(emb_data[cluster_labels==i].T[0],emb_data[cluster_labels==i].T[1], label=classes[i], c=colors_cycle[i])
plt.legend(ncol=4, fontsize=10)
#plt.xlim(red_data.T[0].min(),red_data.T[0].max())
plt.tick_params(labelsize=20)
plt.show()
fig.savefig("tsne_02.pdf")

## TM

In [ ]:
tm_data = pd.read_csv("tm/tm_level_0_topic-dist.csv").set_index('doc').reindex(index=files).drop('i_doc',1).fillna(0).values

In [ ]:
fig=plt.figure(figsize=(15,15))
for i in range(cluster_labels.max()+1):
    plt.scatter(tm_data[cluster_labels==i].T[0],tm_data[cluster_labels==i].T[2], label=classes[i], c=colors_cycle[i])
plt.legend(ncol=4, fontsize=10)
#plt.xlim(red_data.T[0].min(),red_data.T[0].max())
plt.tick_params(labelsize=20)
plt.show()
fig.savefig("tm_02.pdf")

In [ ]:
from master_thesis.hsbmpy import out_to_file

In [ ]:
out_to_file(cluster_labels, files, name='lda')

In [ ]:
df_file = pd.read_csv("https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_v7_Annotations_SampleAttributesDS.txt", sep='\t').loc[:,['SAMPID','SMTS', 'SMTSD']]
df_file.set_index('SAMPID', inplace=True)

In [ ]:
df_file.reindex(index=files).to_csv("files.dat")